In [179]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____


# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [93]:
# import PyPDF2
# import re
# import win32com.client
# from win32com.client.dynamic import Dispatch
import os
from pdf2image import convert_from_path
# import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
# import numpy as np
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [94]:

# FUnction to extract words from a pdf and save it in as a text file

def pdf_2_txt(path):          
    for file in os.listdir(path): # goes through the folders'
        if file.endswith('.pdf'): # finds pdfs
            p = os.path.join(path,file) # grabs path of pdf
            pdfpages = convert_from_path(p, 500) # Converting each page in the pdf into image file with 500 dpi
            image_counter = 1
            for page in pdfpages:
                filename = f"page_{image_counter}.jpg"
                # Save the image of the page in system
                page.save(filename, 'JPEG')
                image_counter+=1
                text = str(((pytesseract.image_to_string(Image.open(filename))))) # takes image and process into text
                text = text.replace('-\n', '') # some text processing
                # delete image
                os.remove(filename)

                filename = filename.replace('.jpg','') # Naming
                with open(f'{filename}.txt', 'w+') as f: # Opens a text file and writes the characters onto
                    print(text, file=f)
                    print(f'Page_{image_counter}.txt has been saved into {os.getcwd()}')
                
    return
                    
                
        


In [158]:

# path = os.path.join(os.getcwd(),'syllabus.pdf')
path = os.getcwd()
pdf_2_txt(path)
# pages = convert_from_path(path, 500) 
# for page in pages:
#     page.save('test.jpg')

Page_2.txt has been saved into C:\Users\15713\Desktop
Page_3.txt has been saved into C:\Users\15713\Desktop
Page_4.txt has been saved into C:\Users\15713\Desktop
Page_5.txt has been saved into C:\Users\15713\Desktop
Page_6.txt has been saved into C:\Users\15713\Desktop
Page_7.txt has been saved into C:\Users\15713\Desktop
Page_8.txt has been saved into C:\Users\15713\Desktop
Page_9.txt has been saved into C:\Users\15713\Desktop
Page_10.txt has been saved into C:\Users\15713\Desktop


In [49]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib

# % matplotlib inline

In [51]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords 

In [176]:
def combine_pages(path):
    # Combining pages
    pages = ''
    for file in os.listdir(path):
        if file.endswith('.txt'):
            with codecs.open(file, 'r') as f: 
                page = f.read()
                pages+=page
    return pages

# pages = combine_pages(path)
# pages

In [144]:
# text = pages
# tokens_P = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
# tokens = WordPunctTokenizer().tokenize(text)


In [137]:
# getting a list of unecessary words like 'the', 'or',... to avoid
avoid = stopwords.words('english')
avoid.append('edu') # adding our own word to the list

In [111]:
word_pattern = re.compile("\w+$") # finds \w: all word characters, $: end of string character

In [142]:
def get_text_counter(text, avoid, word_pattern):
#     tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = WordPunctTokenizer().tokenize(text)
    tokens = list(map(lambda x: x.lower(), tokens)) # converting everything to lower-case
    # cleaning up tokens by getting rid of stopwords and finding word characters
    tokens = [token for token in tokens if re.match(word_pattern, token) and token not in avoid] 
    # collections.counter() returns the tokens and their frequency as a dictionary ({freq: token})
    return collections.Counter(tokens), len(tokens) 

In [168]:
def make_df(counter, size):
    abs_freq = np.array([x[1] for x in counter]) # the absolute freqz is found from the collections.counter()
    rel_freq = abs_freq/size # relative freqz is calculated by dividing abs_freqz by the size
    index = [x[0] for x in counter] # the tokens are used as an index 
    # creating the dataframe
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, 
                      index=index,
                      columns=["Absolute frequency","Relative frequency"])
    # Naming the index title
    df.index.name = "Least Common Words"
    return df

In [177]:
path = os.getcwd()

pages = combine_pages(path) # combining every text file in folder
word_count, size = get_text_counter(pages, avoid, word_pattern) # returns tokens freqz and count of tokens
n = 1000 # 1000 of least commonly occuring words
# least commonly occuring words are chosen with the assumption that a syllabus will only mention
# technical terms once or twice while other words like 'students' and so on could be redundant
df = make_df(word_count.most_common()[:-n-1:-1], size) # finds the least commonly occuring words


In [178]:
df.head(50)

,Absolute frequency,Relative frequency
Least Common Words,,
ittp,1.0,0.000478
4111,1.0,0.000478
hot,1.0,0.000478
tip,1.0,0.000478
anonymous,1.0,0.000478
solvers,1.0,0.000478
reporting,1.0,0.000478
2810,1.0,0.000478
call,1.0,0.000478


In [160]:
n = 1000
print([x[0] for x in word_count.most_common()[:-n-1:-1]])

['sunday', '45am', '45pm', 'semester', 'come', 'requests', 'missing', 'entertain', 'returned', 'report', 'grades', 'cyse220_cw02_poppinsmary', 'cyse220_hwk02_doejohn', 'firstname', 'cyse220_assigmenttypeandweek_lastname', 'convention', 'named', 'files', 'explained', 'challenge', 'quiz', 'test', 'prior', 'disclosed', 'amount', 'conducted', 'challenges', 'tests', 'day', 'usually', 'end', '50', '70', 'max', 'earn', 'submit', 'handed', 'beginning', 'present', 'otherwise', 'stated', 'unless', '100', 'types', 'three', 'hard', 'shows', 'consistently', 'past', 'subject', 'understanding', 'insight', 'gain', 'detail', 'sometimes', 'date', 'follows', 'structure', '05', 'pack', 'service', 'v', '1259012051', '1259012050', '2012', 'ii', '0073529271', '0073529273', '848', '2009', 'january', '0073534870', '2010', 'also', 'encouraged', '0073398068', '0073398063', '45s', '928', '2013', 'march', 'textbook', 'friday', 'cosine', 'sine', 'blocks', 'variable', 'state', 'diagram', 'block', 'asynchronous', 'de

In [149]:
df.tail(50)
# word_count

,Absolute frequency,Relative frequency
Most Common Words,,
subject,1.0,0.00088
past,1.0,0.00088
consistently,1.0,0.00088
shows,1.0,0.00088
hard,1.0,0.00088
three,1.0,0.00088
types,1.0,0.00088
100,1.0,0.00088
unless,1.0,0.00088
